# SQLAlchemy

## What you will learn in this course 🧐🧐

If you want to manipulate Databases, it is good for you to know the Python library: `sql-alchemy`.

In this course, you'll learn: 

- Read & Write Data with SQL commands
- Read data from your datalake and load it to a proper database 
- Read data from your database and load it to datalake 
- Use Pandas features to read & write sql database

## Introduction to SQLAlchemy 🧙‍♂️🧙‍♂️

SQLAlchemy is the best way to handle relational databases using Python code. As you grow in your data career, you should know the basic principles as you will be handling databases.

### Structure of the API 

SQLAlchemy is organized in layers:

![](https://docs.sqlalchemy.org/en/13/_images/sqla_arch_small.png)

You have two ways of handling SQL databases with SQLAlchemy: 

- ORM: It stands for Object Relation Mapper where you'll be able to communicate with the database and create very flexible models.
- SQLAlchemy Core: This layer is more schema-centric and will allow you to very simply access and query your tables. 

Let's check-out core functionnalities of both these layers.

### Nota Bene ➤ SQLlite

For the rest of this course, we'll be using SQLlite which is a very light version of a SQL database that is built-in. Therefore, we won't have to create any production database like PostgreSQL or MySQL. However, please note that SQLlite is NOT made for production and that you will be using other kind of DB (like PostgreSQL or MySQL) in a production environment.

### ORM 

ORM uses python classes and instances to create and manipulate databases. Let's see how it works. Before diving into creating tables, let's create a db by connecting to it.

#### Create a connection

In [ ]:
## Libraries to install if other connections than sqlite
# !pip install pymysql # For MySQL engines
# !pip install psycopg2-binary # For PostgreSQL engines

In [2]:
# Import sqlalchemy
from sqlalchemy import create_engine, text

# Create engine will create a connection between a SQLlite DB and python
 engine = create_engine("sqlite:///:memory:", echo=True)
# engine = create_engine(f"mysql+pymysql://{DBUSER}:{DBPASS}@{DBHOST}:{PORT}/{DBNAME}", echo=True)
# engine = create_engine(f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOSTNAME}/{DBNAME}", echo=True)

In the above code, we created a "fake" database where we'll be using our computer's memory to mimic what a real database would be. 

> NB: if you were to use a production database such as PostgreSQL, you would be using a connection just like this: `postgresql+psycopg2://{dbuser}:{dbpass}@{dbhost}/{dbname}`
>
> NB: if you were to use a MySQL production database, you would be using a connection just like this: `mysql+pymysql://{DBUSER}:{DBPASS}@{DBHOST}:{PORT}/{DBNAME}`

where you would need to specify: 

```python
DBHOST = "HOST_FROM_AMAZON_RDS"
DBUSER = "USERNAME"
DBPASS = "PASSWORD"
DBNAME = "DBNAME"
PORT = "PORT"
# DBNAME = "postgres" --> If you are using PostgreSQL
```

#### Create a table 

Let's now create a table. To do so, we'll be using a python class and instanciate it. 

In [3]:
# Let's instanciate a declarative base to be able to use our python class
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Let's define our table using a class
from sqlalchemy import Column, Integer, String

class User(Base):
    __tablename__ = "users"

    # Each parameter corresponds to a column in our DB table
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)

    def __repr__(self):
        return "<User(name='{}', fullname='{}', nickname='{}')>".format(self.name, self.fullname, self.nickname)

Here we represented our table `users` by class. As you can see, it contains 4 columns: 

- `id` of type Integer
- `name` of type String 
- `fullname` of type String 
- `nickname` of type String 

The `__repr__` method simply states how the output will be formated when we'll be calling our attributes.

Now, we need to create our table within our database. We can do this by using the `create_all` method from `Base.metadata` module. 

In [4]:
Base.metadata.create_all(engine)

2021-03-16 15:22:18,674 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-03-16 15:22:18,679 INFO sqlalchemy.engine.base.Engine ()
2021-03-16 15:22:18,681 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-03-16 15:22:18,683 INFO sqlalchemy.engine.base.Engine ()
2021-03-16 15:22:18,684 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("users")
2021-03-16 15:22:18,686 INFO sqlalchemy.engine.base.Engine ()
2021-03-16 15:22:18,687 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("users")
2021-03-16 15:22:18,688 INFO sqlalchemy.engine.base.Engine ()
2021-03-16 15:22:18,690 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


2021-03-16 15:22:18,691 INFO sqlalchemy.engine.base.Engine ()
2021-03-16 15:22:18,694 INFO sqlalchemy.engine.base.Engine COMMIT


#### Insert values 

Let's insert values in our database:

In [5]:
# Create a new instance of User will allow us to insert a new record later on
ed_user = User(id=1, name='ed', fullname='Ed Jones', nickname='edsnickname')

# Access Full row 
print(ed_user)

# Access ed_user name 
name = ed_user.name
print("name: {}".format(name))

# Access ed_user nickname
nickname = ed_user.nickname 
print("nickname: {}".format(nickname))

<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>
name: ed
nickname: edsnickname


We created data! As you can see we can access each column values simply by declaring a `.column_name`.

#### Persist values in db

Eventhough we created values, we haven't saved it in our database. We can do it by opening a `Session`:

In [6]:
# Initialize a sessionmaker 
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)

Here we created a `sessionmaker` which will allow us to talk to our database. The `bind` argument takes an `engine` as parameter which corresponds to our database.

In [7]:
# Create a new instance of User will allow us to insert a new record later on
al_user = User(id=2, name='al', fullname='Al Jones', nickname='alsnickname')

# Access Full row 
print(al_user)

<User(name='al', fullname='Al Jones', nickname='alsnickname')>


In [8]:
# Instanciate Session 
session = Session()

# Add values to db 
session.add(ed_user)
session.add(al_user)

# Commit the results 
session.commit()

2021-03-16 15:22:22,768 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-03-16 15:22:22,771 INFO sqlalchemy.engine.base.Engine INSERT INTO users (id, name, fullname, nickname) VALUES (?, ?, ?, ?)
2021-03-16 15:22:22,773 INFO sqlalchemy.engine.base.Engine ((1, 'ed', 'Ed Jones', 'edsnickname'), (2, 'al', 'Al Jones', 'alsnickname'))
2021-03-16 15:22:22,774 INFO sqlalchemy.engine.base.Engine COMMIT


Good job! We added our first value inside our db 👏 Note that it is very important to use the `.commit()` method to actually persist the values you inserted when you called the `.add()` method.

#### Query values from a database

Now that we have some data inside our database, we can query it simply by using the `session` instance.

In [9]:
# Query our table users
user = session.query(User)

# Output all the results 
user.all()

2021-03-16 15:22:53,437 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-03-16 15:22:53,470 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users
2021-03-16 15:22:53,471 INFO sqlalchemy.engine.base.Engine ()


[<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>,
 <User(name='al', fullname='Al Jones', nickname='alsnickname')>]

You can now also use any SQL statement if you want to run more complex queries:

In [11]:
from sqlalchemy import text

# Create a statement 
statement = text("SELECT * FROM users where name=:name")
statement

As you can see, the query looks almost as a real SQL query. The only thing is the `:name` parameter in `where name=:name`.

This will allow us to insert values afterwards the following way: 

In [12]:
session.query(User).from_statement(statement).params(name="ed").all()

2021-03-16 15:23:53,546 INFO sqlalchemy.engine.base.Engine SELECT * FROM users where name=?
2021-03-16 15:23:53,547 INFO sqlalchemy.engine.base.Engine ('ed',)


[<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>]

As you can see, we used this `params` method that allows you to specify an operator from which you could apply your filter.

### SQLAlchemy Core 

If you don't like classes and the declarative approach. You can use the SQLAlchemy Core layer of the library. Let's start by creating a new database. 

In [13]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)

#### Create Tables 

You can simply create tables the following way.

In [14]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

meta = MetaData()

# Define table "students"
students = Table(
    'students', meta, 
    Column('id', Integer, primary_key = True), 
    Column('name', String), 
    Column('lastname', String),
)

# Define table "adresses"
addresses = Table(
    'addresses', meta, 
    Column('id', Integer, primary_key = True), 
    Column('email_address', String), 
    Column("student_id", None, ForeignKey("students.id"))
)

As you can see, I simply declared two tables with the `Table` class. Pay attention to `meta` as well which basically contain all the metadata (additional information) necessary to create an actual table. 

Let's now `create_all` tables to actually create them within our database. 

In [15]:
meta.create_all(engine)

2021-03-16 15:23:59,762 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-03-16 15:23:59,763 INFO sqlalchemy.engine.base.Engine ()
2021-03-16 15:23:59,765 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-03-16 15:23:59,766 INFO sqlalchemy.engine.base.Engine ()
2021-03-16 15:23:59,768 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("students")
2021-03-16 15:23:59,769 INFO sqlalchemy.engine.base.Engine ()
2021-03-16 15:23:59,771 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("students")
2021-03-16 15:23:59,772 INFO sqlalchemy.engine.base.Engine ()
2021-03-16 15:23:59,773 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("addresses")
2021-03-16 15:23:59,774 INFO sqlalchemy.engine.base.Engine ()
2021-03-16 15:23:59,775 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("addresses")
2021-03-16 15:23:59,776 INFO sqlalchemy.engine.base.Engine ()
2021-03-16 1

#### Insert values

If you need to insert values, you can do it the following way: 

In [16]:
ins = students.insert().values(id="1", name="Jack", lastname="Johnson")
ins

This hasn't really inserted values just yet. You will need to first:

1. Create a connecion to your db 
2. Execute the query

In [17]:
# Connect to the db
conn = engine.connect()

# Execute the query 
result = conn.execute(ins)
result

2021-03-16 15:24:04,548 INFO sqlalchemy.engine.base.Engine INSERT INTO students (id, name, lastname) VALUES (?, ?, ?)
2021-03-16 15:24:04,549 INFO sqlalchemy.engine.base.Engine ('1', 'Jack', 'Johnson')
2021-03-16 15:24:04,551 INFO sqlalchemy.engine.base.Engine COMMIT


If you need to insert multiple values, you can do it simply by specifying a list of dictionnaries:

In [18]:
values = [
    {'student_id': 1, 'email_address' : 'jack@yahoo.com'},
    {'student_id': 1, 'email_address' : 'jack@msn.com'}
]

conn.execute(addresses.insert(), values)

2021-03-16 15:24:06,949 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, student_id) VALUES (?, ?)
2021-03-16 15:24:06,951 INFO sqlalchemy.engine.base.Engine (('jack@yahoo.com', 1), ('jack@msn.com', 1))
2021-03-16 15:24:06,952 INFO sqlalchemy.engine.base.Engine COMMIT


#### Query values 

If you need to query your database, you can do it via `text` method and use actual sql queries. 

In [19]:
from sqlalchemy.sql import text

# Create a statement 
stmt = text("SELECT students.id, addresses.id, students.name, addresses.email_address FROM students "
            "JOIN addresses ON students.id=addresses.student_id "
            "WHERE students.id = 1")

result = conn.execute(stmt)

2021-03-16 15:24:08,685 INFO sqlalchemy.engine.base.Engine SELECT students.id, addresses.id, students.name, addresses.email_address FROM students JOIN addresses ON students.id=addresses.student_id WHERE students.id = 1
2021-03-16 15:24:08,687 INFO sqlalchemy.engine.base.Engine ()


What is important to notice is: 

- Each line of the query is represented by a string. If you query needs to be on several lines, you need to add a space (" ") before going to the next line.
- The `.columns()` works like a `.format()` method. This is where you will be specifying the names of the columns, you want to insert.
- Finally, for each columns we specified `table_name.c.column_name`. This is how would select a column from a given table.

Now if you want to check the actual results. You can use the `.fetchall` method that will output the results as a list. 

In [20]:
result.fetchall()

[(1, 1, 'Jack', 'jack@yahoo.com'), (1, 2, 'Jack', 'jack@msn.com')]

## Simple things out with Pandas 😌

SQLAlchemy is the fundamental knowledge when dealing with SQL databases with Python as it's broadly used by web-development libraries and others. However, there are simpler way to insert and query databases that would work most of the time: Pandas.

### Read SQL Databases with Pandas 

Most of the time, you were using pandas with CSV files or Excel spreadsheets but you can also use this library with SQL databases. You would use the `read_sql` method. 

In [21]:
import pandas as pd 

# Create a statement 
# Within the text() method is a SQL query. Check out our SQL reminder course if you feel a little rusty
stmt = text("SELECT students.id, addresses.id, students.name, addresses.email_address FROM students "
            "JOIN addresses ON students.id=addresses.student_id "
            "WHERE students.id = 1")

# Create a dataFrame using SQL
df = pd.read_sql(
        stmt,
        engine
    )

df.head()

2021-03-16 15:25:11,974 INFO sqlalchemy.engine.base.OptionEngine SELECT students.id, addresses.id, students.name, addresses.email_address FROM students JOIN addresses ON students.id=addresses.student_id WHERE students.id = 1
2021-03-16 15:25:11,975 INFO sqlalchemy.engine.base.OptionEngine ()


,id,id,name,email_address
0,1,1,Jack,jack@yahoo.com
1,1,2,Jack,jack@msn.com


Now you have a very nice DataFrame that you can easily manipulate! 

Note that we still used SQLAlchemy as we create a statement `stmt` as well as an `engine` for pandas to be able to create it. 

### Update your database with Pandas 

The same way you can read sql, you can also write sql with pandas. 

In [22]:
# Create a new column
df["great_new_column"] = 0
df.head()

,id,id,name,email_address,great_new_column
0,1,1,Jack,jack@yahoo.com,0
1,1,2,Jack,jack@msn.com,0


In [23]:
# Push this new dataframe to our sql database
df.to_sql(
    "brand_new_table",
    engine
)

2021-03-16 15:25:24,419 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("brand_new_table")
2021-03-16 15:25:24,420 INFO sqlalchemy.engine.base.Engine ()
2021-03-16 15:25:24,423 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("brand_new_table")
2021-03-16 15:25:24,424 INFO sqlalchemy.engine.base.Engine ()
2021-03-16 15:25:24,427 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE brand_new_table (
	"index" BIGINT, 
	id BIGINT, 
	name TEXT, 
	email_address TEXT, 
	great_new_column BIGINT
)


2021-03-16 15:25:24,429 INFO sqlalchemy.engine.base.Engine ()
2021-03-16 15:25:24,430 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-16 15:25:24,431 INFO sqlalchemy.engine.base.Engine CREATE INDEX ix_brand_new_table_index ON brand_new_table ("index")
2021-03-16 15:25:24,433 INFO sqlalchemy.engine.base.Engine ()
2021-03-16 15:25:24,435 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-16 15:25:24,439 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-03-16 15:25:24,441 INFO s

In [24]:
# Let's query it 
stmt = text("SELECT * FROM brand_new_table")
result = conn.execute(stmt)
result.fetchall()

2021-03-16 15:25:50,204 INFO sqlalchemy.engine.base.Engine SELECT * FROM brand_new_table
2021-03-16 15:25:50,205 INFO sqlalchemy.engine.base.Engine ()


[(0, 1, 'Jack', 'jack@yahoo.com', 0), (1, 2, 'Jack', 'jack@msn.com', 0)]

## Resources 📚📚

- <a href="https://www.youtube.com/watch?v=woKYyhLCcnU" target="_blank">Introduction to SQLAlchemy</a>
- <a href="https://docs.sqlalchemy.org/en/13/intro.html" target="_blank">SQLAlchemy Overview</a>
- <a href="https://docs.sqlalchemy.org/en/13/orm/tutorial.html" target="_blank">Object Relational Tutorial</a>
- <a href="https://docs.sqlalchemy.org/en/13/core/tutorial.html" target="_blank">SQL Expression Language Tutorial</a>
- <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql.html#pandas.read_sql" target="_blank">Pandas Read SQL</a>
- <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html" target="_blank">Pandas To SQL</a>